In [0]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

In [0]:
# Загрузим набор данных

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
Exposure             115155 non-null float64
LicAge               115155 non-null int64
RecordBeg            115155 non-null object
RecordEnd            59455 non-null object
Gender               115155 non-null object
MariStat             115155 non-null object
SocioCateg           115155 non-null object
VehUsage             115155 non-null object
DrivAge              115155 non-null int64
HasKmLimit           115155 non-null int64
BonusMalus           115155 non-null int64
ClaimAmount          115155 non-null float64
ClaimInd             115155 non-null int64
ClaimNbResp          115155 non-null float64
ClaimNbNonResp       115155 non-null float64
ClaimNbParking       115155 non-null float64
ClaimNbFireTheft     115155 non-null float64
ClaimNbWindscreen    115155 non-null float64
OutUseNb             115155 non-null float64
RiskArea             115155 non-null float64
dtypes

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [0]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [0]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [0]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [0]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [0]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

**NB**: В H2O не рекомендуется использовать one-hot encoding, поскольку данный фреймворк корректно работает с категориальными признаками, тогда как применение one-hot encoding приводит к неэффективности. Тем не менее, используем здесь фиктивные переменные, чтобы в дальнейшем сохранить возможность сравнения результатов построенных моделей.

In [0]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [0]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [0]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [0]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [0]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [0]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [0]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [0]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,VehUsage_Private,VehUsage_Private+trip to office,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0,0,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0,0,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,0,1,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0,1,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1,0,0,0,0,0,0,0,0,1,0,3249


### Установка H2O на Google Colaboratory и инициализация

In [0]:
!apt-get install default-jre

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
!java -version

openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)


In [0]:
!pip install h2o

In [0]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpb1bu9huh
  JVM stdout: /tmp/tmpb1bu9huh/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpb1bu9huh/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_unknownUser_po2suk
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


## * Домашнее задание: GLM для прогнозирования наступления страхового случая

In [0]:
# Разбиение датасета на train/val/test

from sklearn.model_selection import train_test_split
x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1) # stratify=df.ClaimInd
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1) # stratify=y_test_ind

In [0]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

for x in [h2o_train_ind, h2o_valid_ind, h2o_test_ind]:
    x['ClaimInd'] = x['ClaimInd'].asfactor()

In [0]:
h2o_train_ind.names[1:-1]

['LicAge',
 'Gender',
 'MariStat',
 'DrivAge',
 'HasKmLimit',
 'BonusMalus',
 'ClaimNbResp',
 'ClaimNbNonResp',
 'ClaimNbParking',
 'ClaimNbFireTheft',
 'ClaimNbWindscreen',
 'OutUseNb',
 'RiskArea',
 'VehUsage_Private',
 'VehUsage_Private+trip to office',
 'VehUsage_Professional',
 'VehUsage_Professional run',
 'SocioCateg_CSP1',
 'SocioCateg_CSP2',
 'SocioCateg_CSP3',
 'SocioCateg_CSP4',
 'SocioCateg_CSP5',
 'SocioCateg_CSP6',
 'SocioCateg_CSP7',
 'DrivAgeSq']

In [0]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_binomial = H2OGeneralizedLinearEstimator(family = "binomial", link = "Logit", nfolds=5)
glm_binomial.train(y="ClaimInd", x = h2o_train_ind.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [0]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_binomial.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 2.368E-5 )",25,25,3,py_1_sid_95c8


In [0]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_binomial.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.5152063,0.053481974,0.4886589,0.5285016,0.50821817,0.59711117,0.45354176
1,auc,0.57437015,0.0036078044,0.5739018,0.5732919,0.5778384,0.57769924,0.56911933
2,aucpr,0.1177733,0.002612848,0.116333455,0.118587896,0.12177087,0.11729222,0.11488208
3,err,0.4847937,0.053481974,0.5113411,0.47149843,0.49178183,0.40288883,0.54645824
4,err_count,7815.4,859.91876,8251.0,7618.0,7899.0,6499.0,8810.0
5,f0point5,0.13707864,0.0054911263,0.13113847,0.1412285,0.14280784,0.13881879,0.13139963
6,f1,0.193083,0.0060628513,0.18685326,0.19776748,0.20091046,0.19156611,0.18831767
7,f2,0.32691854,0.01120653,0.32488006,0.3297977,0.33872288,0.30896395,0.33222806
8,lift_top_group,1.584417,0.21699414,1.8158902,1.4518323,1.3795145,1.8239484,1.4508995
9,logloss,0.31054142,0.008043579,0.303693,0.31754658,0.32035026,0.30264708,0.3084702


In [0]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_binomial._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.494727,-2.286852
1,LicAge,-0.000384,-0.061396
2,Gender,0.014988,0.007265
3,MariStat,-0.057466,-0.020702
4,DrivAge,-0.003957,-0.059336
5,HasKmLimit,-0.335855,-0.104884
6,BonusMalus,0.004713,0.072342
7,ClaimNbResp,0.068884,0.036203
8,ClaimNbNonResp,0.134218,0.080457
9,ClaimNbParking,0.172365,0.050834


In [0]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_binomial.coef_norm()
for x in range(len(glm_binomial.cross_validation_models())):
    pmodels[x] = glm_binomial.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-2.28685,-2.27834,-2.29631,-2.30032,-2.27639,-2.28445
LicAge,-0.06140,-0.07443,-0.06655,-0.07359,-0.06387,-0.03073
Gender,0.00726,0.01108,0.00821,-0.00206,0.01379,0.00449
MariStat,-0.02070,-0.02182,0.00078,-0.02002,-0.03597,-0.02207
DrivAge,-0.05934,-0.04319,-0.03625,-0.04087,-0.01621,-0.01822
HasKmLimit,-0.10488,-0.11648,-0.10373,-0.10426,-0.10177,-0.09626
BonusMalus,0.07234,0.06479,0.06546,0.06499,0.07752,0.09585
ClaimNbResp,0.03620,0.03702,0.04548,0.04052,0.03524,0.02059
ClaimNbNonResp,0.08046,0.08275,0.08097,0.07750,0.07311,0.08762
ClaimNbParking,0.05083,0.05121,0.05379,0.05150,0.05467,0.04252


In [0]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_binomial.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_binomial.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_binomial.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [0]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [0]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

print(accuracy_score(y_train_ind, (ind_train_pred.predict).astype(np.int64)))
print(f1_score(y_train_ind, (ind_train_pred.predict).astype(np.int64)))
print(confusion_matrix(y_train_ind, (ind_train_pred.predict).astype(np.int64)))

0.6114653632393807
0.190953475756245
[[45593 27380]
 [ 3939  3696]]


Какие проблемы вы здесь видите? Как можно улучшить данный результат?

Видим, что классы сильно несбалансированы. Поэтому, можно было бы прибегнуть к процедурам бутстрапа, over- и undersamling, использовать стратифицированную выборку и т.д. 

Однако, встроенные в h2o методы для GLM не позволяют "из коробки" сбалансировать датасет (хотя другие методы в h2o их поддерживают). К тому же, есть большое подозрение, что использование нелинейных алгоритмов могло бы помочь улучшить результат.

Также, важно понимать важность такого понятия как feature engineering. Так, если рассмотреть экстремальный случай, если бы мы не исключили переменную `ClaimAmount` из нашего рассмотрения, то мы бы получили практически идеальный классификатор, но стоит заметить, что в таком случае мы бы слили (leak) зависимую переменную, а в реальности подобная задача не имела бы смысла.

В качестве эксперимента с обобщенными линейными моделями попробуем задать различные веса для наблюдений каждого из классов при обучении. Вряд ли это может дать прирост качества классификатора, но может помочь снизить количество ложноотрицательных срабатываний. Потенциально, это может быть оправдано в задачах, когда ошибка первого рода ведет к значительно большим потерям, чем ошибка второго рода.

In [0]:
df['Weight'] = np.where(df.ClaimInd == 1, 2, 1)

In [0]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1)

In [0]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

for x in [h2o_train_ind, h2o_valid_ind, h2o_test_ind]:
    x['ClaimInd'] = x['ClaimInd'].asfactor()

In [0]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_binomial = H2OGeneralizedLinearEstimator(family = "binomial", link = "Logit", nfolds=5)
glm_binomial.train(y="ClaimInd", x = h2o_train_ind.names[1:-2], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind, weights_column='Weight')

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [0]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_binomial.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_binomial.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_binomial.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [0]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

print(accuracy_score(y_train_ind, (ind_train_pred.predict).astype(np.int64)))
print(f1_score(y_train_ind, (ind_train_pred.predict).astype(np.int64)))
print(confusion_matrix(y_train_ind, (ind_train_pred.predict).astype(np.int64)))

0.39553146089718144
0.19003607227754046
[[26167 46806]
 [ 1919  5716]]


Видим, что теперь большая часть наблюдений классифицируется как 1, при более низком показателе accuracy score и почти неизменной f-мерой.

Отметим, что природа прогнозируемого процесса (наступление страхового случая) имеет стохастический характер. И, в отличие от других показателей (как, например, размер убытка или частота), с высокой точностью не представляется возможным предсказание самого факта свершения страхового случая (иначе и не было бы смысла в страховании в принципе).